# **SQL Database REBUILD & REORGANIZE**

- Carlos Eduardo Gimenes
- Last Modified: December, 2023

> **Copyright (C) 2023 Carlos Eduardo Gimenes**  
> All rights reserved.  
> You may alter this code for your own _non-commercial_ purposes.  
> You may republish altered code as long as you include this copyright and give due credit.
> 
> THIS CODE AND INFORMATION ARE PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE IMPLIED WARRANTIES OF MERCHANTABILITY AND/OR FITNESS FOR A PARTICULAR PURPOSE.

## **Pré Requisitos**

- **Database Mail deve estar configurado e funcionando**
    - ver: **[Database Mail](https://github.com/carlosgimenes/Training-Tarefas_Dia_Dia_de_um_DBA/blob/9ed5e8db026432b1083b85ff6e02abde48322329/src/Database-Mail.ipynb)**
- **Database Traces deve existir**
    - ver: **[Database Baseline Whoisactive](https://github.com/carlosgimenes/Training-Tarefas_Dia_Dia_de_um_DBA/blob/9ed5e8db026432b1083b85ff6e02abde48322329/src/DatabaseBaselineWhoisactive.ipynb)**
- **View vwHistorico\_Fragmentacao\_Indice deve existir**
    - ver: **[Database Fragmentação Índices](https://github.com/carlosgimenes/Training-Tarefas_Dia_Dia_de_um_DBA/blob/9ed5e8db026432b1083b85ff6e02abde48322329/src/DatabaseFragmentacaoIndices.ipynb)**

# <span style="font-size: 28px;"><b>Step 1 - Creating Procedure that processes REBUILD or REORGANIZE</b></span>

- **Procedure** <span style="font-size: 14px;"><b>stpManutencao_Indices</b></span>

In [ ]:
/*
-------------------------------------------------------------------------------------------------------------------------
	Title		:	REBUILD or REORGANIZE Index
-------------------------------------------------------------------------------------------------------------------------
	Author		:	Gimenes
	Date		:	14/12/2023
	Requester	:	Gimenes
	Purpose		:	Database Reconstruction or Reorganization of Indexes - Creation Procedure [stpManutencao_Indices]
	Program		:	Not applicable
    Credito     :   http://www.fabriciolima.net/
			
-------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

-----------------------------------------------------------------------------------------------------------------------
-- Procedure to REBUILD or REORGANIZE Index
-----------------------------------------------------------------------------------------------------------------------

USE [Traces]
GO

IF object_id('stpManutencao_Indices') IS NOT NULL
    DROP PROCEDURE stpManutencao_Indices
GO

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

CREATE PROCEDURE [dbo].[stpManutencao_Indices]
AS
BEGIN
    SET LOCK_TIMEOUT 300000 -- Se ficar bloqueado por mais de 5 minutos, aborta.

    DECLARE 
		@Id INT
        , @SQLString NVARCHAR(1000)

    IF OBJECT_ID('tempdb..#Indices_Fragmentados') IS NOT NULL
        DROP TABLE #Indices_Fragmentados

    --	Seleciona os indices fragmentados
    SELECT identity(INT, 1, 1) Id
        , 'ALTER INDEX [' + Nm_Indice + '] ON ' + Nm_Database + '.' + Nm_Schema + '.[' + Nm_Tabela + 
		CASE 
            WHEN Avg_Fragmentation_In_Percent < 30
                THEN '] REORGANIZE'
            ELSE '] REBUILD'											-- Como existe janela para isso, sempre fazer REBUILD
        END Comando
        , Page_Count
        , Nm_Database
        , Nm_Tabela
        , Nm_Indice
        , Fl_Compressao
        , Avg_Fragmentation_In_Percent
    INTO #Indices_Fragmentados
    FROM Traces.dbo.vwHistorico_Fragmentacao_Indice A WITH (NOLOCK) 	-- Tabela que armazena o Historico de Fragmentacao
    JOIN master.sys.databases B
        ON B.name = A.Nm_Database
    WHERE Dt_Referencia >= CAST(FLOOR(cast(getdate() AS FLOAT)) AS DATETIME)
        AND Avg_Fragmentation_In_Percent >= 10
        AND Page_Count > 1000
        AND Nm_Indice IS NOT NULL
        AND B.state_desc = 'ONLINE'

    WHILE EXISTS (
            SELECT Id
            FROM #Indices_Fragmentados
            )
    BEGIN
        SELECT TOP 1 @Id = Id
            , @SQLString = Comando
        FROM #Indices_Fragmentados
        ORDER BY Nm_Database
            , Page_Count

        -- Realiza o REORGANIZE OU O REBUILD
        EXECUTE sp_executesql @SQLString

        DELETE
        FROM #Indices_Fragmentados
        WHERE Id = @Id
    END
END

-- <<< End of Query


# <span style="font-size: 28px;"><b>Step 2 - Creation of the Job that processes REBUILD or REORGANIZE</b></span>

- **Job \[DBA - REBUILD or REORGANIZE Index**\]
    - _Processa aos Sábados_
- _**ATENÇÃO AOS AJUSTES QUE DEVEM SER FEITOS ANTES DE PROCESSAR!!!**_

In [ ]:
/*
--------------------------------------------------------------------------------------------------------------------------------
Title		:	REBUILD or REORGANIZE Index
--------------------------------------------------------------------------------------------------------------------------------
Author		:	Gimenes
Date		:	14/12/2023
Requester	:	Gimenes
Purpose		:	Database Reconstruction or Reorganize Indexes - Create Job [DBA - REBUILD or REORGANIZE Index]
Program		:	Not applicable
Credito     :   http://www.fabriciolima.net/
--------------------------------------------------------------------------------------------------------------------------------
*/

-- >>> Start of Query

USE [msdb]
GO
DECLARE @jobId BINARY(16)
EXEC  msdb.dbo.sp_add_job @job_name=N'DBA - REBUILD or REORGANIZE Index', 
		@enabled=0, 															-- ATENCAO: NASCE DESABILITADO!!!
		@notify_level_eventlog=0, 
		@notify_level_email=2, 
		@notify_level_page=2, 
		@delete_level=0, 
		@description=N'Processamento REBUILD ou REORGANIZE dos Indices', 
		@category_name=N'Database Maintenance', 
		@owner_login_name=N'NOME_USUARIO',			-- AJUSTAR 
		@notify_email_operator_name=N'Alerta_BD', @job_id = @jobId OUTPUT
select @jobId
GO
EXEC msdb.dbo.sp_add_jobserver @job_name=N'DBA - REBUILD or REORGANIZE Index', @server_name = N'NOME_SERVIDOR'  -- AJUSTAR
GO
USE [msdb]
GO
EXEC msdb.dbo.sp_add_jobstep @job_name=N'DBA - REBUILD or REORGANIZE Index', @step_name=N'Step_1', 
		@step_id=1, 
		@cmdexec_success_code=0, 
		@on_success_action=1, 
		@on_fail_action=2, 
		@retry_attempts=0, 
		@retry_interval=0, 
		@os_run_priority=0, @subsystem=N'TSQL', 
		@command=N'EXEC [dbo].[stpManutencao_Indices]', 
		@database_name=N'Traces', 
		@flags=0
GO
USE [msdb]
GO
EXEC msdb.dbo.sp_update_job @job_name=N'DBA - REBUILD or REORGANIZE Index', 
		@enabled=1, 
		@start_step_id=1, 
		@notify_level_eventlog=0, 
		@notify_level_email=2, 
		@notify_level_page=2, 
		@delete_level=0, 
		@description=N'Processamento REBUILD ou REORGANIZE dos Indices', 
		@category_name=N'Database Maintenance', 
		@owner_login_name=N'sa', 
		@notify_email_operator_name=N'Alerta_BD', 
		@notify_page_operator_name=N''
GO
USE [msdb]
GO
DECLARE @schedule_id int
EXEC msdb.dbo.sp_add_jobschedule @job_name=N'DBA - REBUILD or REORGANIZE Index', @name=N'Schedule_1', 
		@enabled=1, 
		@freq_type=8, 
		@freq_interval=64, 
		@freq_subday_type=1, 
		@freq_subday_interval=0, 
		@freq_relative_interval=0, 
		@freq_recurrence_factor=1, 
		@active_start_date=20231208, 
		@active_end_date=99991231, 
		@active_start_time=30000, 
		@active_end_time=235959, @schedule_id = @schedule_id OUTPUT
select @schedule_id
GO

-- <<< End of Query


* * *

# <span style="font-size: 28px;"><b>End of file</b></span>